In [1]:
# Import required libraries  
import os
import textwrap
from openai import AzureOpenAI, DefaultHttpxClient
from dotenv import load_dotenv, find_dotenv
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector

# Set Up

In [2]:
# Configure environment variables  
load_dotenv(find_dotenv('credential.env'), override=True)

# Azure AI Search
service_endpoint = os.environ['AZURE_AI_SEARCH_ENDPOINT']
key = os.environ['AZURE_AI_SEARCH_KEY']
index_name = os.environ['AZURE_AI_SEARCH_INDEX_NAME']
credential = AzureKeyCredential(key)

#Azure OpenAI
client = AzureOpenAI(
  api_key = os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  azure_endpoint = os.environ['AZURE_OPENAI_API_ENDPOINT'],
  api_version = os.environ['AZURE_OPENAI_API_VERSION'],
  http_client=DefaultHttpxClient(verify=False)
)
embedding_model = os.environ['EMBEDDING_MODEL_NAME']
# gpt35turbo_model = os.environ['GPT35TURBO_MODEL_NAME']
# gpt4turbo_model = os.environ['GPT4TURBO_MODEL_NAME']
gpt4o_model = os.environ['GPT4O_MODEL_NAME']

In [3]:
# Declare useful method
def check_and_create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"The folder '{folder_name}' has been created.")
    else:
        print(f"The folder '{folder_name}' already exists.")

def print_error_message(message, prefix_message='Error: '):
    print(f"\033[1;31m{prefix_message}\033[0m{message}")

def print_warning_message(message, prefix_message='Warning: '):
    print(f"\033[1;33m{prefix_message}\033[0m{message}")
    
def print_success_message(message, prefix_message='Success: '):
    print(f"\033[1;32m{prefix_message}\033[0m{message}")

In [4]:
def generate_embeddings(text):
    response = client.embeddings.create(input=text, model=embedding_model)
    return response.data[0].embedding


def get_chat_completion(messages, model):
    print(f'Generating answer using {model} LLM model')
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=800

    )
    return response.choices[0].message.content

def format_retrieved_documents(data_list):
    retrieved_documents = ""
    for item in data_list:
        # Extract the 'content' and 'sourcepage' values
        content = item.get('content', '')
        sourcepage = item.get('sourcepage', '')
        retrieved_documents += "sourcepage: " + sourcepage + ", content: " + content + "\n"
    return retrieved_documents

# Query

In [5]:
query = input("What would you like to ask.")
print(query)

ช่วยบอกเกี่ยวกับประเภทของน้ำมันที่ exxon mobil มีหน่อย


## AI search

In [6]:

search_client = SearchClient(
    service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    select=["content", "sourcepage"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

result_list = []
# Retrieve top k relevance chunked document from Azure AI Search
for result in results:
    print_warning_message(f"{result['sourcepage']}", "Source Page: ")
    print_warning_message(f"{result['@search.score']}", "Search Score: ")
    print_warning_message(f"{textwrap.shorten(result['content'], width=100, placeholder='...')}", "Content: ")
    print_warning_message(f"{result['@search.reranker_score']}", "Reranker Score: ")
    result_list.append(result)

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print_warning_message(f"{caption.highlights}\n", "Caption: ")
        else:
            print_warning_message(f"{caption.text}\n", "Caption: ")

# Generate semantic answers based on top k relevance chunked documents
semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print_warning_message(f"{answer.highlights}", "Semantic Answer: ")
    else:
        print_warning_message(f"{answer.text}", "Semantic Answer: ")
    print_warning_message(f"{answer.score}\n", "Semantic Answer Score: ")

print_success_message("", "-----AI Search JSON Output [Semantic Hybrid Search with reranking]-----")
result_list

/Users/bhanujaya/Developer/Exxon3/GenUniverse/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Source Page: mobil-super-all-in-one-protection-5w-40-0.pdf
Search Score: 0.01315789483487606
Content: เราขอแนะนำน้ำมัน Mobil Super 3000 All-In-One Protection...
Reranker Score: 2.291781425476074
Caption: <em>เราขอแนะนำน้ำมัน Mobil Sup</em>er 3000 All-In-One Protection โดยเฉพาะเมื่อคุณต้องพบกับเงื่อนไขการใช้งานที่ต้องการการหล่อลื่นอย่างสม่ำเสมอ ซึ่งน้ำมันชนิดนี้สามารถช่วยต้านทานความเสียหายที่เกิดจากความเค้นเครื่องยนต์สูงที่มักเกิดขึ้น ;: · เทคโนโลยีเครื่องยนต์เกือบทุกระบบ · เหมาะทั้งสำหรับเครื่องยนต์เบนซินและดีเซล · รถยนต์นั่ง รถ SUV รถบรรทุกเบา และรถตู้ · ก...

Source Page: mobil-1-0.pdf
Search Score: 0.0117647061124444
Content: Mobil 1 TM สูตรใหม่ Triple Action Power+ สมรรถนะเครื่องยนต์ ลดการสึกหรอ...
Reranker Score: 2.2473576068878174
Caption: Triple Action Power+ เป็นน้ำมันเครื่องสังเคราะห์แท้ขั้นสูงที่มอบสมรรถนะ การปกป้อง และความสะอาดของเครื่องยนต์ที่ยอดเยี่ยม พร้อมทั้งประโยชน์เพิ่มเติมด้านการประหยัดเชื้อเพลิง · สมรรถนะเครื่องยนต์: ทรงพลังยาวนาน • ปกป้อง: ดีกว่า 30 เท่า1ภายใต้ความร

[{'sourcepage': 'mobil-super-all-in-one-protection-5w-40-0.pdf',
  'content': 'เราขอแนะนำน้ำมัน Mobil Super 3000 All-In-One Protection โดยเฉพาะเมื่อคุณต้องพบกับเงื่อนไขการใช้งานที่ต้องการการหล่อลื่นอย่างสม่ำเสมอ ซึ่งน้ำมันชนิดนี้สามารถช่วยต้านทานความเสียหายที่เกิดจากความเค้นเครื่องยนต์สูงที่มักเกิดขึ้น ;:\n· เทคโนโลยีเครื่องยนต์เกือบทุกระบบ\n· เหมาะทั้งสำหรับเครื่องยนต์เบนซินและดีเซล\n· รถยนต์นั่ง รถ SUV รถบรรทุกเบา และรถตู้\n· การควบคุมความเร็วบนทางหลวงและการขับขี่ในเมืองแบบหยุดแล้วไป\n· สภาวะการทำงานระดับปานกลางจนถึงบ่อยครั งที่รุนแรง\n• เครื่องยนต์เทอร์โบชาร์จและไดเร็กอินเจ็คชัน\n• เครื่องยนต์สมรรถนะสูง\nให้ศึกษาในคู่มือสำหรับผู้ใช้ยานยนต์เพื่อตรวจสอบเกรดความหนืดที่แนะนำและข้อกำหนดสำหรับยานพาหนะเฉพาะของคุณเสมอ\nข้อกำหนดและการรับรอง\nผลิตภัณฑ์นี้ได้รับการแนะนำสำหรับการใช้งานที่ต้องการ:\nAPI CF\nAPI CF-4\nAPI CG-4\nผลิตภัณฑ์นี้ผ่านหรือเกินกว่าข้อกำหนดของ:\nAPI SM\nACEA A3/B3\nAPI SN\nAPI SN PLUS\nACEA A3/B4\nคุณสมบัติและคุณลักษณะที่กำหนด\nคุณสมบัติ\nเกรด\nSAE 5W-40\nMini-Rotary Viscom

# Prompt

In [7]:
system_prompt = f"""
## On your role
- You are a chatbot for ExxonMobil named Mobilly, designed to help answer customer questions based on retrieved documents and relevant knowledge to refine the answers.
- You always respond in Thai.
- Do not include greetings in your responses.
- Be polite and answer as a kind, helpful assistant.
- Mention the source of your answers.

## Retrieved documents
{format_retrieved_documents(result_list)}

## Instructions
- Only answer questions related to the topics covered in the retrieved documents.
- If a question is outside the scope of the retrieved documents, try to answer based on your knowledge.
"""
print(system_prompt)



## On your role
- You are a chatbot for ExxonMobil named Mobilly, designed to help answer customer questions based on retrieved documents and relevant knowledge to refine the answers.
- You always respond in Thai.
- Do not include greetings in your responses.
- Be polite and answer as a kind, helpful assistant.
- Mention the source of your answers.

## Retrieved documents
sourcepage: mobil-super-all-in-one-protection-5w-40-0.pdf, content: เราขอแนะนำน้ำมัน Mobil Super 3000 All-In-One Protection โดยเฉพาะเมื่อคุณต้องพบกับเงื่อนไขการใช้งานที่ต้องการการหล่อลื่นอย่างสม่ำเสมอ ซึ่งน้ำมันชนิดนี้สามารถช่วยต้านทานความเสียหายที่เกิดจากความเค้นเครื่องยนต์สูงที่มักเกิดขึ้น ;:
· เทคโนโลยีเครื่องยนต์เกือบทุกระบบ
· เหมาะทั้งสำหรับเครื่องยนต์เบนซินและดีเซล
· รถยนต์นั่ง รถ SUV รถบรรทุกเบา และรถตู้
· การควบคุมความเร็วบนทางหลวงและการขับขี่ในเมืองแบบหยุดแล้วไป
· สภาวะการทำงานระดับปานกลางจนถึงบ่อยครั งที่รุนแรง
• เครื่องยนต์เทอร์โบชาร์จและไดเร็กอินเจ็คชัน
• เครื่องยนต์สมรรถนะสูง
ให้ศึกษาในคู่มือสำหรับผู้ใช้

## GEMMA 2

In [9]:
# !pip install google-generativeai
# !pip install google-ai-generativelanguage

In [10]:
import google
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["API_KEY"])

model = genai.GenerativeModel('gemini-1.5-flash')

/Users/bhanujaya/Developer/Exxon3/GenUniverse/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model = genai.GenerativeModel('gemini-1.5-flash',
                              generation_config={"response_mime_type": "application/json"})

prompt = query + system_prompt

response = model.generate_content(prompt)


In [12]:
print(response.text)

{"น้ำมันหล่อลื่นของเอ็กซอนโมบิลมีหลายประเภท ขึ้นอยู่กับความต้องการของเครื่องยนต์และการใช้งาน คุณสามารถดูข้อมูลเพิ่มเติมได้จากคู่มือผู้ใช้ยานยนต์ของคุณ": "Mobil Super 3000 All-In-One Protection เป็นน้ำมันเครื่องที่ออกแบบมาสำหรับเครื่องยนต์ทุกประเภท ทั้งเบนซินและดีเซล เหมาะสำหรับรถยนต์นั่ง รถ SUV รถบรรทุกเบา และรถตู้  นอกจากนี้ยังมีน้ำมันเครื่อง Mobil 1 ซึ่งแบ่งออกเป็น 2 รุ่น คือ Mobil 1 Triple Action Power+ และ Mobil 1 ESP  น้ำมันเครื่อง Mobil 1 Triple Action Power+ เป็นน้ำมันเครื่องสังเคราะห์แท้ขั้นสูงที่มอบสมรรถนะ การปกป้อง และความสะอาดของเครื่องยนต์ที่ยอดเยี่ยม  ส่วน Mobil 1 ESP เป็นน้ำมันเครื่องที่ออกแบบมาเพื่อช่วยเพิ่มประสิทธิภาพของเครื่องยนต์และลดการปล่อยมลพิษ  นอกจากนี้ยังมี Mobil 1 Turbo Diesel 5W-40 ซึ่งเป็นน้ำมันเครื่องสังเคราะห์แท้ขั้นสูงที่ออกแบบมาเพื่อให้การปกป้องและสมรรถนะสูงสุดสำหรับเครื่องยนต์ดีเซล คุณสามารถดูข้อมูลเพิ่มเติมเกี่ยวกับน้ำมันเครื่องของเอ็กซอนโมบิลได้จากเอกสารที่คุณให้มา  "}



In [16]:
ณซ.

SyntaxError: invalid syntax (2419068954.py, line 1)